In [13]:
import pickle
from tqdm import tqdm
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sentence_transformers import SentenceTransformer
from rank_paraphrases import rank_paraphrases

In [30]:
#constants
lm_checkpoint = 's-nlp/t5-paraphrase-paws-msrp-opinosis-paranmt'
model_name = lm_checkpoint.split('/')[1]
cls_checkpoint = 's-nlp/roberta_toxicity_classifier'

In [23]:
with open('test_1k_toxic.txt') as f:
    toxic_inputs = f.read().splitlines()

In [7]:
with open('caif-t5-paraphrase-paws-msrp-opinosis-paranmt_test_1k.pickle', 'rb') as f:
    samples1 = pickle.load(f)

with open('caif-t5-paraphrase-paws-msrp-opinosis-paranmt_test_1k2.pickle', 'rb') as f:
    samples2 = pickle.load(f)

In [10]:
samples = samples1 + samples2

In [19]:
style_cls_model = AutoModelForSequenceClassification.from_pretrained(cls_checkpoint)
style_tokenizer = AutoTokenizer.from_pretrained(cls_checkpoint)
style_cls_model.cuda();

Some weights of the model checkpoint at s-nlp/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
sim_model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')

In [29]:
para_results = []
best_candidates = []

for i, candidates in tqdm(zip(toxic_inputs, samples)):
    ranked_candidates = rank_paraphrases(
        candidates,
        i,
        style_cls_model,
        style_tokenizer,
        sim_model,
        style_score_threshold=0.99
    )
    para_results.append(ranked_candidates['ranked_candidates'])
    best_candidates.append(ranked_candidates['best_candidate'][2])

1000it [00:38, 25.70it/s]


In [31]:
with open(f'caif-{model_name}-5-samples_test_1k.txt', 'w', encoding='utf-8') as file:
    file.writelines([sentence + '\n' for sentence in best_candidates])